# Introduction to Financial Python
## Pandas-Resampling and DataFrame
### Introduction
In the last chapter we had a glimpse of Pandas. In this chapter we will learn about resampling methods and the DataFrame object, which is a powerful tool for financial data analysis.

### Fetching Data
Here we use the Quandl API to retrieve data.

In [25]:
import pandas_datareader as pdr
import pandas as pd
import numpy as np

We will create a Series named "aapl" whose values are Apple's daily closing prices, which are of course indexed by dates:

In [2]:
start = "2016-01-01"
end = "2018-01-01"
aapl_table = pdr.DataReader('AAPL', 'yahoo', start, end)
aapl = aapl_table['Adj Close']['2017']
print (aapl)

Date
2017-01-03    27.459938
2017-01-04    27.429203
2017-01-05    27.568691
2017-01-06    27.876030
2017-01-09    28.131361
                ...    
2017-12-22    42.040852
2017-12-26    40.974274
2017-12-27    40.981483
2017-12-28    41.096786
2017-12-29    40.652374
Name: Adj Close, Length: 251, dtype: float64


Recall that we can fetch a specific data point using series['yyyy-mm-dd']. We can also fetch the data in a specific month using series['yyyy-mm'].

In [3]:
print (aapl['2017-3'])

Date
2017-03-01    33.192142
2017-03-02    32.995064
2017-03-03    33.189762
2017-03-06    33.085289
2017-03-07    33.128029
2017-03-08    33.004562
2017-03-09    32.928577
2017-03-10    33.037796
2017-03-13    33.052044
2017-03-14    33.002182
2017-03-15    33.351227
2017-03-16    33.405842
2017-03-17    33.239635
2017-03-20    33.588676
2017-03-21    33.204002
2017-03-22    33.579170
2017-03-23    33.460449
2017-03-24    33.393963
2017-03-27    33.450958
2017-03-28    34.144287
2017-03-29    34.220264
2017-03-30    34.175156
2017-03-31    34.111042
Name: Adj Close, dtype: float64


Or in several consecutive months:

In [4]:
aapl['2017-2':'2017-4']

Date
2017-02-01    30.438799
2017-02-02    30.386791
2017-02-03    30.516821
2017-02-06    30.802887
2017-02-07    31.096045
                ...    
2017-04-24    34.106297
2017-04-25    34.317619
2017-04-26    34.115788
2017-04-27    34.141903
2017-04-28    34.108673
Name: Adj Close, Length: 61, dtype: float64

.head(N) and .tail(N) are methods for quickly accessing the first or last N elements.

In [5]:
print (aapl.head())
print (aapl.tail(10))

Date
2017-01-03    27.459938
2017-01-04    27.429203
2017-01-05    27.568691
2017-01-06    27.876030
2017-01-09    28.131361
Name: Adj Close, dtype: float64
Date
2017-12-15    41.791023
2017-12-18    42.379566
2017-12-19    41.927948
2017-12-20    41.882305
2017-12-21    42.040852
2017-12-22    42.040852
2017-12-26    40.974274
2017-12-27    40.981483
2017-12-28    41.096786
2017-12-29    40.652374
Name: Adj Close, dtype: float64


## Resampling
**series.resample(freq)** is a class called "DatetimeIndexResampler" which groups data in a Series object into regular time intervals. The argument "freq" determines the length of each interval.

**series.resample.mean()** is a complete statement that groups data into intervals, and then compute the mean of each interval. For example, if we want to aggregate the daily data into monthly data by mean:



In [6]:
by_month = aapl.resample('M').mean()
print (by_month)

Date
2017-01-31    28.268486
2017-02-28    31.707397
2017-03-31    33.388701
2017-04-30    33.927464
2017-05-31    36.241263
2017-06-30    35.246389
2017-07-31    35.358004
2017-08-31    38.018475
2017-09-30    37.725050
2017-10-31    37.775021
2017-11-30    41.365839
2017-12-31    41.291726
Freq: M, Name: Adj Close, dtype: float64


We can also aggregate the data by week:

In [7]:
by_week = aapl.resample('W').mean()
print (by_week.head())

Date
2017-01-08    27.583466
2017-01-15    28.187630
2017-01-22    28.356554
2017-01-29    28.645335
2017-02-05    29.757445
Freq: W-SUN, Name: Adj Close, dtype: float64


We can choose almost any frequency by using the format 'nf', where 'n' is an integer and 'f' is M for month, W for week and D for day.

In [8]:
three_day = aapl.resample('3D').mean()
two_week  = aapl.resample('2W').mean()
two_month = aapl.resample('2M').mean()

Besides the mean() method, other methods can also be used with the resampler:

In [9]:
std = aapl.resample('W').std()    # standard deviation
max = aapl.resample('W').max()    # maximum value
min = aapl.resample('W').min()    # minimum value

Often we want to calculate monthly returns of a stock, based on prices on the last day of each month. To fetch those prices, we use the series.resample.agg() method:

In [10]:
last_day = aapl.resample('M').agg(lambda x: x[-1])
print (last_day)

Date
2017-01-31    28.689308
2017-02-28    32.527302
2017-03-31    34.111042
2017-04-30    34.108673
2017-05-31    36.421490
2017-06-30    34.337669
2017-07-31    35.460644
2017-08-31    39.254917
2017-09-30    36.890041
2017-10-31    40.461285
2017-11-30    41.281757
2017-12-31    40.652374
Freq: M, Name: Adj Close, dtype: float64


Or directly calculate the monthly rates of return using the data for the first day and the last day:

In [11]:
monthly_return = aapl.resample('M').agg(lambda x: x[-1]/x[1] - 1)
print (monthly_return)

Date
2017-01-31    0.045940
2017-02-28    0.070442
2017-03-31    0.033823
2017-04-30   -0.007736
2017-05-31    0.039865
2017-06-30   -0.073529
2017-07-31    0.032202
2017-08-31    0.047754
2017-09-30   -0.049112
2017-10-31    0.094252
2017-11-30    0.025922
2017-12-31   -0.003357
Freq: M, Name: Adj Close, dtype: float64


Series object also provides us some convenient methods to do some quick calculation.

In [12]:
print (monthly_return.mean())
print (monthly_return.std())
print (monthly_return.max())

0.0213722266581254
0.04775585142622976
0.09425164908662276


Another two methods frequently used on Series are .diff() and .pct_change(). The former calculates the difference between consecutive elements, and the latter calculates the percentage change.

In [13]:
print (last_day.diff())
print (last_day.pct_change())

Date
2017-01-31         NaN
2017-02-28    3.837994
2017-03-31    1.583740
2017-04-30   -0.002369
2017-05-31    2.312817
2017-06-30   -2.083820
2017-07-31    1.122974
2017-08-31    3.794273
2017-09-30   -2.364876
2017-10-31    3.571243
2017-11-30    0.820473
2017-12-31   -0.629383
Freq: M, Name: Adj Close, dtype: float64
Date
2017-01-31         NaN
2017-02-28    0.133778
2017-03-31    0.048690
2017-04-30   -0.000069
2017-05-31    0.067807
2017-06-30   -0.057214
2017-07-31    0.032704
2017-08-31    0.107000
2017-09-30   -0.060244
2017-10-31    0.096808
2017-11-30    0.020278
2017-12-31   -0.015246
Freq: M, Name: Adj Close, dtype: float64


Notice that we induced a NaN value while calculating percentage changes i.e. returns.

When dealing with NaN values, we usually either removing the data point or fill it with a specific value. Here we fill it with 0:

In [14]:
daily_return = last_day.pct_change()
print (daily_return.fillna(0))

Date
2017-01-31    0.000000
2017-02-28    0.133778
2017-03-31    0.048690
2017-04-30   -0.000069
2017-05-31    0.067807
2017-06-30   -0.057214
2017-07-31    0.032704
2017-08-31    0.107000
2017-09-30   -0.060244
2017-10-31    0.096808
2017-11-30    0.020278
2017-12-31   -0.015246
Freq: M, Name: Adj Close, dtype: float64


Alternatively, we can fill a NaN with the next fitted value. This is called 'backward fill', or 'bfill' in short:

In [15]:
daily_return = last_day.pct_change()
print (daily_return.fillna(method = 'bfill'))

Date
2017-01-31    0.133778
2017-02-28    0.133778
2017-03-31    0.048690
2017-04-30   -0.000069
2017-05-31    0.067807
2017-06-30   -0.057214
2017-07-31    0.032704
2017-08-31    0.107000
2017-09-30   -0.060244
2017-10-31    0.096808
2017-11-30    0.020278
2017-12-31   -0.015246
Freq: M, Name: Adj Close, dtype: float64


As expected, since there is a 'backward fill' method, there must be a 'forward fill' method, or 'ffill' in short. However we can't use it here because the NaN is the first value.

We can also simply remove NaN values by .dropna()

In [16]:
daily_return = last_day.pct_change().dropna()
print (daily_return)

Date
2017-02-28    0.133778
2017-03-31    0.048690
2017-04-30   -0.000069
2017-05-31    0.067807
2017-06-30   -0.057214
2017-07-31    0.032704
2017-08-31    0.107000
2017-09-30   -0.060244
2017-10-31    0.096808
2017-11-30    0.020278
2017-12-31   -0.015246
Freq: M, Name: Adj Close, dtype: float64


## DataFrame
The DataFrame is the most commonly used data structure in Pandas. It is essentially a table, just like an Excel spreadsheet.

More precisely, a DataFrame is a collection of Series objects, each of which may contain different data types. A DataFrame can be created from various data types: dictionary, 2-D numpy.ndarray, a Series or another DataFrame.


### Create DataFrames
The most common method of creating a DataFrame is passing a dictionary:

In [17]:
dict = {'AAPL': [143.5,  144.09, 142.73, 144.18, 143.77],
        'GOOG': [898.7,  911.71, 906.69, 918.59, 926.99],
        'IBM':  [155.58, 153.67, 152.36, 152.94, 153.49]}
dates = pd.date_range('2017-07-03', periods = 5, freq = 'D')
df = pd.DataFrame(dict, index = dates)
print (df)

              AAPL    GOOG     IBM
2017-07-03  143.50  898.70  155.58
2017-07-04  144.09  911.71  153.67
2017-07-05  142.73  906.69  152.36
2017-07-06  144.18  918.59  152.94
2017-07-07  143.77  926.99  153.49


### Manipulating DataFrames
We can fetch values in a DataFrame by columns and index. Each column in a DataFrame is essentially a Pandas Series. We can fetch a column by square brackets: **df['column_name']**

If a column name contains no spaces, then we can also use df.column_name to fetch a column:

In [18]:
df = aapl_table
print (df.Close.tail(5))
print (df['Volume'].tail(5))

Date
2017-12-22    43.752499
2017-12-26    42.642502
2017-12-27    42.650002
2017-12-28    42.770000
2017-12-29    42.307499
Name: Close, dtype: float64
Date
2017-12-22     65397600.0
2017-12-26    132742000.0
2017-12-27     85992800.0
2017-12-28     65920800.0
2017-12-29    103999600.0
Name: Volume, dtype: float64


All the methods we applied to a Series index such as iloc[], loc[] and resampling methods, can also be applied to a DataFrame:

In [19]:
aapl_2016 = df['2016']
aapl_month = aapl_2016.resample('M').agg(lambda x: x[-1])
print (aapl_month)

                 High        Low       Open      Close       Volume  Adj Close
Date                                                                          
2016-01-31  24.334999  23.587500  23.697500  24.334999  257666000.0  22.512041
2016-02-29  24.557501  24.162500  24.215000  24.172501  140865200.0  22.483057
2016-03-31  27.475000  27.219999  27.430000  27.247499  103553600.0  25.343138
2016-04-30  23.680000  23.127501  23.497499  23.434999  274126000.0  21.797100
2016-05-31  25.100000  24.705000  24.900000  24.965000  169228800.0  23.361542
2016-06-30  23.942499  23.575001  23.610001  23.900000  143345600.0  22.364946
2016-07-31  26.137501  25.920000  26.047501  26.052500  110934800.0  24.379189
2016-08-31  26.642500  26.410000  26.415001  26.525000  118649600.0  24.955809
2016-09-30  28.342501  27.950001  28.115000  28.262501  145516400.0  26.590519
2016-10-31  28.557501  28.299999  28.412500  28.385000  105677600.0  26.705769
2016-11-30  28.049999  27.567499  27.900000  27.6299

We may select certain columns of a DataFrame using their names:

In [21]:
aapl_bar = aapl_month[['Open', 'High', 'Low', 'Close']]
print (aapl_bar)

                 Open       High        Low      Close
Date                                                  
2016-01-31  23.697500  24.334999  23.587500  24.334999
2016-02-29  24.215000  24.557501  24.162500  24.172501
2016-03-31  27.430000  27.475000  27.219999  27.247499
2016-04-30  23.497499  23.680000  23.127501  23.434999
2016-05-31  24.900000  25.100000  24.705000  24.965000
2016-06-30  23.610001  23.942499  23.575001  23.900000
2016-07-31  26.047501  26.137501  25.920000  26.052500
2016-08-31  26.415001  26.642500  26.410000  26.525000
2016-09-30  28.115000  28.342501  27.950001  28.262501
2016-10-31  28.412500  28.557501  28.299999  28.385000
2016-11-30  27.900000  28.049999  27.567499  27.629999
2016-12-31  29.162500  29.299999  28.857500  28.955000


We can even specify both rows and columns using loc[]. The row indices and column names are separated by a comma:

In [23]:
print (aapl_month.loc['2016-03':'2016-06', ['Open', 'High', 'Low', 'Close']])

                 Open       High        Low      Close
Date                                                  
2016-03-31  27.430000  27.475000  27.219999  27.247499
2016-04-30  23.497499  23.680000  23.127501  23.434999
2016-05-31  24.900000  25.100000  24.705000  24.965000
2016-06-30  23.610001  23.942499  23.575001  23.900000


The subset methods in DataFrame is quite useful. By writing logical statements in square brackets, we can make customized subsets:

In [26]:
above = aapl_bar[aapl_bar.Close > np.mean(aapl_bar.Close)]
print (above)

                 Open       High        Low      Close
Date                                                  
2016-03-31  27.430000  27.475000  27.219999  27.247499
2016-08-31  26.415001  26.642500  26.410000  26.525000
2016-09-30  28.115000  28.342501  27.950001  28.262501
2016-10-31  28.412500  28.557501  28.299999  28.385000
2016-11-30  27.900000  28.049999  27.567499  27.629999
2016-12-31  29.162500  29.299999  28.857500  28.955000


### Data Validation
As mentioned, all methods that apply to a Series can also be applied to a DataFrame. Here we add a new column to an existing DataFrame:

In [27]:
aapl_bar['rate_return'] = aapl_bar.Close.pct_change()
print (aapl_bar)

                 Open       High        Low      Close  rate_return
Date                                                               
2016-01-31  23.697500  24.334999  23.587500  24.334999          NaN
2016-02-29  24.215000  24.557501  24.162500  24.172501    -0.006678
2016-03-31  27.430000  27.475000  27.219999  27.247499     0.127211
2016-04-30  23.497499  23.680000  23.127501  23.434999    -0.139921
2016-05-31  24.900000  25.100000  24.705000  24.965000     0.065287
2016-06-30  23.610001  23.942499  23.575001  23.900000    -0.042660
2016-07-31  26.047501  26.137501  25.920000  26.052500     0.090063
2016-08-31  26.415001  26.642500  26.410000  26.525000     0.018136
2016-09-30  28.115000  28.342501  27.950001  28.262501     0.065504
2016-10-31  28.412500  28.557501  28.299999  28.385000     0.004334
2016-11-30  27.900000  28.049999  27.567499  27.629999    -0.026599
2016-12-31  29.162500  29.299999  28.857500  28.955000     0.047955


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Here the calculation introduced a NaN value. If the DataFrame is large, we would not be able to observe it. isnull() provides a convenient way to check abnormal values.

In [28]:
missing = aapl_bar.isnull()
print (missing)
print ('---------------------------------------------')
print (missing.describe())

             Open   High    Low  Close  rate_return
Date                                               
2016-01-31  False  False  False  False         True
2016-02-29  False  False  False  False        False
2016-03-31  False  False  False  False        False
2016-04-30  False  False  False  False        False
2016-05-31  False  False  False  False        False
2016-06-30  False  False  False  False        False
2016-07-31  False  False  False  False        False
2016-08-31  False  False  False  False        False
2016-09-30  False  False  False  False        False
2016-10-31  False  False  False  False        False
2016-11-30  False  False  False  False        False
2016-12-31  False  False  False  False        False
---------------------------------------------
         Open   High    Low  Close rate_return
count      12     12     12     12          12
unique      1      1      1      1           2
top     False  False  False  False       False
freq       12     12     12     12    

The row labelled "unique" indicates the number of unique values in each column. Since the "rate_return" column has 2 unique values, it has at least one missing value.

We can deduce the number of missing values by comparing "count" with "freq". There are 12 counts and 11 False values, so there is one True value which corresponds to the missing value.

We can also find the rows with missing values easily:

In [29]:
print (missing[missing.rate_return == True])

             Open   High    Low  Close  rate_return
Date                                               
2016-01-31  False  False  False  False         True


Usually when dealing with missing data, we either delete the whole row or fill it with some value. As we introduced in the Series chapter, the same method **dropna()** and **fillna()** can be applied to a DataFrame.

In [30]:
drop = aapl_bar.dropna()
print (drop)
print ('\n--------------------------------------------------\n')
fill = aapl_bar.fillna(0)
print (fill)

                 Open       High        Low      Close  rate_return
Date                                                               
2016-02-29  24.215000  24.557501  24.162500  24.172501    -0.006678
2016-03-31  27.430000  27.475000  27.219999  27.247499     0.127211
2016-04-30  23.497499  23.680000  23.127501  23.434999    -0.139921
2016-05-31  24.900000  25.100000  24.705000  24.965000     0.065287
2016-06-30  23.610001  23.942499  23.575001  23.900000    -0.042660
2016-07-31  26.047501  26.137501  25.920000  26.052500     0.090063
2016-08-31  26.415001  26.642500  26.410000  26.525000     0.018136
2016-09-30  28.115000  28.342501  27.950001  28.262501     0.065504
2016-10-31  28.412500  28.557501  28.299999  28.385000     0.004334
2016-11-30  27.900000  28.049999  27.567499  27.629999    -0.026599
2016-12-31  29.162500  29.299999  28.857500  28.955000     0.047955

--------------------------------------------------

                 Open       High        Low      Close  rate_re

### DataFrame Concat
We have seen how to extract a Series from a dataFrame. Now we need to consider how to merge a Series or a DataFrame into another one.

In Pandas, the function **concat()** allows us to merge multiple Series into a DataFrame:

In [31]:
s1 = pd.Series([143.5, 144.09, 142.73, 144.18, 143.77], name = 'AAPL')
s2 = pd.Series([898.7, 911.71, 906.69, 918.59, 926.99], name = 'GOOG')
data_frame = pd.concat([s1, s2], axis = 1)
print (data_frame)

     AAPL    GOOG
0  143.50  898.70
1  144.09  911.71
2  142.73  906.69
3  144.18  918.59
4  143.77  926.99


The "axis = 1" parameter will join two DataFrames by columns:

In [32]:
log_price = np.log(aapl_bar.Close)
log_price.name = 'log_price'
print (log_price)
print ('\n--------------------------------------------\n')
concat = pd.concat([aapl_bar, log_price], axis = 1)
print (concat)

Date
2016-01-31    3.191916
2016-02-29    3.185216
2016-03-31    3.304962
2016-04-30    3.154231
2016-05-31    3.217475
2016-06-30    3.173878
2016-07-31    3.260114
2016-08-31    3.278088
2016-09-30    3.341536
2016-10-31    3.345861
2016-11-30    3.318902
2016-12-31    3.365743
Freq: M, Name: log_price, dtype: float64

--------------------------------------------

                 Open       High        Low      Close  rate_return  log_price
Date                                                                          
2016-01-31  23.697500  24.334999  23.587500  24.334999          NaN   3.191916
2016-02-29  24.215000  24.557501  24.162500  24.172501    -0.006678   3.185216
2016-03-31  27.430000  27.475000  27.219999  27.247499     0.127211   3.304962
2016-04-30  23.497499  23.680000  23.127501  23.434999    -0.139921   3.154231
2016-05-31  24.900000  25.100000  24.705000  24.965000     0.065287   3.217475
2016-06-30  23.610001  23.942499  23.575001  23.900000    -0.042660   3.173878

We can also join two DataFrames by rows. Consider these two DataFrames:

In [35]:
df_volume = aapl_table.loc['2016-10':'2017-04', ['Volume', 'Adj Close']].resample('M').agg(lambda x: x[-1])
print (df_volume)
print ('\n-------------------------------------------\n')
df_2017 = aapl_table.loc['2016-10':'2017-04', ['Open', 'High', 'Low', 'Close']].resample('M').agg(lambda x: x[-1])
print (df_2017)

                 Volume  Adj Close
Date                              
2016-10-31  105677600.0  26.705769
2016-11-30  144649200.0  26.128902
2016-12-31  122345200.0  27.381918
2017-01-31  196804000.0  28.689308
2017-02-28   93931600.0  32.527302
2017-03-31   78646800.0  34.111042
2017-04-30   83441600.0  34.108673

-------------------------------------------

                 Open       High        Low      Close
Date                                                  
2016-10-31  28.412500  28.557501  28.299999  28.385000
2016-11-30  27.900000  28.049999  27.567499  27.629999
2016-12-31  29.162500  29.299999  28.857500  28.955000
2017-01-31  30.287500  30.347500  30.155001  30.337500
2017-02-28  34.270000  34.360001  34.174999  34.247501
2017-03-31  35.930000  36.067501  35.752499  35.915001
2017-04-30  36.022499  36.075001  35.817501  35.912498


Now we merge the DataFrames with our DataFrame 'aapl_bar'

In [36]:
concat = pd.concat([aapl_bar, df_volume], axis = 1)
print (concat)

                 Open       High  ...       Volume  Adj Close
Date                              ...                        
2016-01-31  23.697500  24.334999  ...          NaN        NaN
2016-02-29  24.215000  24.557501  ...          NaN        NaN
2016-03-31  27.430000  27.475000  ...          NaN        NaN
2016-04-30  23.497499  23.680000  ...          NaN        NaN
2016-05-31  24.900000  25.100000  ...          NaN        NaN
2016-06-30  23.610001  23.942499  ...          NaN        NaN
2016-07-31  26.047501  26.137501  ...          NaN        NaN
2016-08-31  26.415001  26.642500  ...          NaN        NaN
2016-09-30  28.115000  28.342501  ...          NaN        NaN
2016-10-31  28.412500  28.557501  ...  105677600.0  26.705769
2016-11-30  27.900000  28.049999  ...  144649200.0  26.128902
2016-12-31  29.162500  29.299999  ...  122345200.0  27.381918
2017-01-31        NaN        NaN  ...  196804000.0  28.689308
2017-02-28        NaN        NaN  ...   93931600.0  32.527302
2017-03-

By default the DataFrame are joined with all of the data. This default options results in zero information loss. We can also merge them by intersection, this is called 'inner join':

In [37]:
concat = pd.concat([aapl_bar, df_volume], axis = 1, join = 'inner')
print (concat)

               Open       High        Low  ...  rate_return       Volume  Adj Close
Date                                       ...                                     
2016-10-31  28.4125  28.557501  28.299999  ...     0.004334  105677600.0  26.705769
2016-11-30  27.9000  28.049999  27.567499  ...    -0.026599  144649200.0  26.128902
2016-12-31  29.1625  29.299999  28.857500  ...     0.047955  122345200.0  27.381918

[3 rows x 7 columns]


Only the intersection part was left if use 'inner join' method. Now let's try to append a DataFrame to another one:

In [39]:
append = aapl_bar.append(df_2017)
print (append)

                 Open       High        Low      Close  rate_return
Date                                                               
2016-01-31  23.697500  24.334999  23.587500  24.334999          NaN
2016-02-29  24.215000  24.557501  24.162500  24.172501    -0.006678
2016-03-31  27.430000  27.475000  27.219999  27.247499     0.127211
2016-04-30  23.497499  23.680000  23.127501  23.434999    -0.139921
2016-05-31  24.900000  25.100000  24.705000  24.965000     0.065287
2016-06-30  23.610001  23.942499  23.575001  23.900000    -0.042660
2016-07-31  26.047501  26.137501  25.920000  26.052500     0.090063
2016-08-31  26.415001  26.642500  26.410000  26.525000     0.018136
2016-09-30  28.115000  28.342501  27.950001  28.262501     0.065504
2016-10-31  28.412500  28.557501  28.299999  28.385000     0.004334
2016-11-30  27.900000  28.049999  27.567499  27.629999    -0.026599
2016-12-31  29.162500  29.299999  28.857500  28.955000     0.047955
2016-10-31  28.412500  28.557501  28.299999  28.

'Append' is essentially to concat two DataFrames by axis = 0, thus here is an alternative way to append:

In [41]:
concat = pd.concat([aapl_bar, df_2017], axis = 0)
print (concat)

                 Open       High        Low      Close  rate_return
Date                                                               
2016-01-31  23.697500  24.334999  23.587500  24.334999          NaN
2016-02-29  24.215000  24.557501  24.162500  24.172501    -0.006678
2016-03-31  27.430000  27.475000  27.219999  27.247499     0.127211
2016-04-30  23.497499  23.680000  23.127501  23.434999    -0.139921
2016-05-31  24.900000  25.100000  24.705000  24.965000     0.065287
2016-06-30  23.610001  23.942499  23.575001  23.900000    -0.042660
2016-07-31  26.047501  26.137501  25.920000  26.052500     0.090063
2016-08-31  26.415001  26.642500  26.410000  26.525000     0.018136
2016-09-30  28.115000  28.342501  27.950001  28.262501     0.065504
2016-10-31  28.412500  28.557501  28.299999  28.385000     0.004334
2016-11-30  27.900000  28.049999  27.567499  27.629999    -0.026599
2016-12-31  29.162500  29.299999  28.857500  28.955000     0.047955
2016-10-31  28.412500  28.557501  28.299999  28.

Please note that if the two DataFrame have some columns with the same column names, these columns are considered to be the same and will be merged. It's very important to have the right column names. If we change a column names here:

In [42]:
df_2017.columns = ['Change', 'High', 'Low', 'Close']
concat = pd.concat([aapl_bar, df_2017], axis = 0)
print (concat)

                 Open       High        Low      Close  rate_return     Change
Date                                                                          
2016-01-31  23.697500  24.334999  23.587500  24.334999          NaN        NaN
2016-02-29  24.215000  24.557501  24.162500  24.172501    -0.006678        NaN
2016-03-31  27.430000  27.475000  27.219999  27.247499     0.127211        NaN
2016-04-30  23.497499  23.680000  23.127501  23.434999    -0.139921        NaN
2016-05-31  24.900000  25.100000  24.705000  24.965000     0.065287        NaN
2016-06-30  23.610001  23.942499  23.575001  23.900000    -0.042660        NaN
2016-07-31  26.047501  26.137501  25.920000  26.052500     0.090063        NaN
2016-08-31  26.415001  26.642500  26.410000  26.525000     0.018136        NaN
2016-09-30  28.115000  28.342501  27.950001  28.262501     0.065504        NaN
2016-10-31  28.412500  28.557501  28.299999  28.385000     0.004334        NaN
2016-11-30  27.900000  28.049999  27.567499  27.6299

Since the column name of 'Open' has been changed, the new DataFrame has an new column named 'Change'.

## Summary
Hereby we introduced the most import part of python: resampling and DataFrame manipulation. We only introduced the most commonly used method in Financial data analysis. There are also many methods used in data mining, which are also beneficial. You can always check the Pandas official documentations for help.